In [1]:
import pandas as pd
import datetime as dt
import os
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

In [2]:
cc = pd.read_csv('C:\\Users\\MirekR\\Documents\\Python\\kodilla\\db\\country_codes.csv')
customers = pd.read_csv('C:\\Users\\MirekR\\Documents\\Python\\kodilla\\db\\customers.csv')
prod_info = pd.read_csv('C:\\Users\\MirekR\\Documents\\Python\\kodilla\\db\\prod_cat_info.csv')
src = 'C:\\Users\\MirekR\\Documents\\Python\\kodilla\\db\\transactions'

In [3]:
cc.rename(columns={ 'Unnamed: 0':'country_code'},inplace=True)
cc.reset_index()
cc.set_index('country_code',inplace=True)
transactions = pd.DataFrame()
for filename in os.listdir(src):
    transactions = transactions.append(pd.read_csv(os.path.join(src,filename),index_col=0))
def convert_dates(x):
    try:
        return dt.datetime.strptime(x,'%d-%m-%Y')
    except:
        return dt.datetime.strptime(x,'%d/%m/%Y')
    return transactions
transactions['tran_date'] = transactions['tran_date'].apply(lambda x: convert_dates(x))
df = transactions.join(prod_info.drop_duplicates(subset=['prod_cat_code']).set_index('prod_cat_code')['prod_cat'],on='prod_cat_code',how='left')
df = df.join(prod_info.drop_duplicates(subset=['prod_sub_cat_code']).set_index('prod_sub_cat_code')['prod_subcat'],on='prod_subcat_code',how='left')
df = df.join(customers.join(cc,on='country_code').set_index('customer_Id'),on='cust_id')
df['day']=df['tran_date'].dt.day_name()

C:\Users\MirekR\AppData\Local\Temp\ipykernel_15080\2438258240.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transactions = transactions.append(pd.read_csv(os.path.join(src,filename),index_col=0))
C:\Users\MirekR\AppData\Local\Temp\ipykernel_15080\2438258240.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transactions = transactions.append(pd.read_csv(os.path.join(src,filename),index_col=0))
C:\Users\MirekR\AppData\Local\Temp\ipykernel_15080\2438258240.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transactions = transactions.append(pd.read_csv(os.path.join(src,filename),index_col=0))
C:\Users\MirekR\AppData\Local\Temp\ipykernel_15080\2438258240.py:6: FutureWarning: The frame.append method is deprecated and wi

In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([html.Div([dcc.Tabs(id='tabs',value='tab-1',children=[
                            dcc.Tab(label='Sprzedaż globalna',value='tab-1'),
                            dcc.Tab(label='Produkty',value='tab-2'),
                            dcc.Tab(label='Kanały sprzedaży',value='tab-3')
                            ]),
                            html.Div(id='tabs-content')
                    ],style={'width':'80%','margin':'auto'})],
                    style={'height':'100%'})
@app.callback(Output('tabs-content','children'),[Input('tabs','value')])
def render_content(tab):

    if tab == 'tab-1':
        return tab1_render_tab(df)
    if tab == 'tab-2':
        return tab2_render_tab(df)
    elif tab == 'tab-3':
        return tab3_render_tab(df)
    
## tab1 callbacks
@app.callback(Output('bar-sales','figure'),
    [Input('sales-range','start_date'),Input('sales-range','end_date')])
def tab1_bar_sales(start_date,end_date):

    truncated = df[(df['tran_date']>=start_date)&(df['tran_date']<=end_date)]
    grouped = truncated[truncated['total_amt']>0].groupby([pd.Grouper(key='tran_date',freq='M'),'Store_type'])['total_amt'].sum().round(2).unstack()

    traces = []
    for col in grouped.columns:
        traces.append(go.Bar(x=grouped.index,y=grouped[col],name=col,hoverinfo='text',
        hovertext=[f'{y/1e3:.2f}k' for y in grouped[col].values]))

    data = traces
    fig = go.Figure(data=data,layout=go.Layout(title='Przychody',barmode='stack',legend=dict(x=0,y=-0.5)))

    return fig
@app.callback(Output('choropleth-sales','figure'),
            [Input('sales-range','start_date'),Input('sales-range','end_date')])
def tab1_choropleth_sales(start_date,end_date):

    truncated = df[(df['tran_date']>=start_date)&(df['tran_date']<=end_date)]
    grouped = truncated[truncated['total_amt']>0].groupby('country')['total_amt'].sum().round(2)

    trace0 = go.Choropleth(colorscale='Viridis',reversescale=True,
                            locations=grouped.index,locationmode='country names',
                            z = grouped.values, colorbar=dict(title='Sales'))
    data = [trace0]
    fig = go.Figure(data=data,layout=go.Layout(title='Mapa',geo=dict(showframe=False,projection={'type':'natural earth'})))

    return fig
## tab2 callbacks
@app.callback(Output('barh-prod-subcat','figure'),
            [Input('prod_dropdown','value')])
def tab2_barh_prod_subcat(chosen_cat):

    grouped = df[(df['total_amt']>0)&(df['prod_cat']==chosen_cat)].pivot_table(index='prod_subcat',columns='Gender',values='total_amt',aggfunc='sum').assign(_sum=lambda x: x['F']+x['M']).sort_values(by='_sum').round(2)

    traces = []
    for col in ['F','M']:
        traces.append(go.Bar(x=grouped[col],y=grouped.index,orientation='h',name=col))

    data = traces
    fig = go.Figure(data=data,layout=go.Layout(barmode='stack',margin={'t':20,}))
    return fig

## tab3 callbacks
@app.callback(Output('barh-Store_type','figure'),
            [Input('store_dropdown','value')])
def tab3_barh_Store_type(chosen_value):

    grouped = df[(df['total_amt']>0)&(df['day']==chosen_value)].pivot_table(index='Store_type',columns='Gender',values='total_amt',aggfunc='sum').assign(_sum=lambda x: x['F']+x['M']).sort_values(by='_sum').round(2)
    #grouped = df[(df['total_amt']>0)&(df['Store_type']==chosen_value)].pivot_table(index='Store_type',columns='day',values='total_amt',aggfunc='sum').assign(_sum=lambda x: x['M']+x['T']+x['W']+x['Th']+x['F']+x['Sa']+x['Su']).sort_values(by='_sum').round(2)
    traces = []
    #for col in ['M','T','W','Th','F','Sa','Su']:                                         
    for col in ['F','M']:
        traces.append(go.Bar(x=grouped[col],y=grouped.index,orientation='h',name=col))
    data = traces
    fig = go.Figure(data=data,layout=go.Layout(barmode='stack',margin={'t':20,}))
    return fig
def tab1_render_tab(df):

    layout = html.Div([html.H1('Sprzedaż globalna',style={'text-align':'center'}),
                        html.Div([dcc.DatePickerRange(id='sales-range',
                        start_date=df['tran_date'].min(),
                        end_date=df['tran_date'].max(),
                        display_format='YYYY-MM-DD')],style={'width':'100%','text-align':'center'}),
                        html.Div([html.Div([dcc.Graph(id='bar-sales')],style={'width':'50%'}),
                        html.Div([dcc.Graph(id='choropleth-sales')],style={'width':'50%'})],style={'display':'flex'})
                        ])

    return layout
def tab2_render_tab(df):

    grouped = df[df['total_amt']>0].groupby('prod_cat')['total_amt'].sum()
    fig = go.Figure(data=[go.Pie(labels=grouped.index,values=grouped.values)],layout=go.Layout(title='Udział grup produktów w sprzedaży'))

    layout = html.Div([html.H1('Produkty',style={'text-align':'center'}),

                        html.Div([html.Div([dcc.Graph(id='pie-prod-cat',figure=fig)],style={'width':'50%'}),
                        html.Div([dcc.Dropdown(id='prod_dropdown',
                                    options=[{'label':prod_cat,'value':prod_cat} for prod_cat in df['prod_cat'].unique()],
                                    value=df['prod_cat'].unique()[0]),
                                    dcc.Graph(id='barh-prod-subcat')],style={'width':'50%'})],style={'display':'flex'}),
                                    html.Div(id='temp-out')
                        ])

    return layout

def tab3_render_tab(df):

    grouped = df[df['total_amt']>0].groupby('day')['total_amt'].sum()
    fig = go.Figure(data=[go.Pie(labels=grouped.index,values=grouped.values)],layout=go.Layout(title='Dni sprzedaży'))

    layout = html.Div([html.H1('Kanały sprzedaży',style={'text-align':'center'}),

                        html.Div([html.Div([dcc.Graph(id='pie-prod-cat',figure=fig)],style={'width':'50%'}),
                        html.Div([dcc.Dropdown(id='store_dropdown',
                                    options=[{'label':Store_type,'value':Store_type} for Store_type in df['Store_type'].unique()],
                                    value=df['day'].unique()[0]),
                                    dcc.Graph(id='barh-Store_type')],style={'width':'50%'})],style={'display':'flex'}),
                                    html.Div(id='temp-out')
                        ])

    return layout


if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File ~\anaconda3\lib\site-packages\pandas\core\indexes\base.py:3802, in Index.get_loc(
    self=Index([], dtype='object', name='Gender'),
    key='F',
    method=None,
    tolerance=None
)
   3801 try:
-> 3802     return self._engine.get_loc(casted_key)
        casted_key = 'F'
        self = Index([], dtype='object', name='Gender')
   3803 except KeyError as err:

File ~\anaconda3\lib\site-packages\pandas\_libs\index.pyx:138, in pandas._libs.index.IndexEngine.get_loc()

File ~\anaconda3\lib\site-packages\pandas\_libs\index.pyx:165, in pandas._libs.index.IndexEngine.get_loc()

File pandas\_libs\hashtable_class_helper.pxi:5745, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas\_libs\hashtable_class_helper.pxi:5753, in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'F'

The above exception was the direct 